In [36]:
!pip install transformers

In [37]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [133]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Mysterious_Affair_at_Styles_Train_Data.csv')


In [171]:
split_data.text[6]

'd fortune hunting; butthere you are-she is her own mistress, and she\'s married him.""It must be a difficult situation for you all.""Difficult! It\'s damnable!"Thus it came about that, three days later, I descended from the trainat Styles St. Mary, an absurd little station, with no apparent reasonfor existence, perched up in the midst of green fields and countrylanes. John Cavendish was waiting on the platform, and piloted me outto the car."Got a drop or two of petrol still, you see," he remarked. "Mainlyowing to the mater\'s activities."The village of Styles St. Mary was situated about two miles from thelittle station, and Styles Court lay a mile the other side of it. Itwas a still, warm day in early July. As one looked out over the flatEssex country, lying so green and peaceful under the afternoon sun, itseemed almost impossible to believe that, not so very far away, a greatwar was running its appointed course. I felt I had suddenly strayedinto another world. As we turned in at the 

In [135]:
split_data = pd.DataFrame(columns=['text', 'OrginalPage'])

In [136]:
# data=data.sample(frac=1)
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [137]:
for _, row in data.iterrows():
    # Split the text into two equal parts
    text1, text2 = row['Text'][len(row['Text']) // 3:2*len(row['Text']) //2 ], row['Text'][:]
    # Add the split texts to the dataframe as separate rows
    split_data = split_data.append({'text': text1, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
    split_data = split_data.append({'text': text2, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
    
# split_data=split_data.sample(frac=1)
# Print the resulting dataframe
split_data

,text,OrginalPage
0,"whole story. This, we trust, will effectually...",1
1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...,1
2,"y. Mrs. Cavendish, who\nhad married John's fat...",2
3,"looked his forty-five years. As a boy, though,...",2
4,rtained a shrewd suspicion that he would have ...,3
...,...,...
409,something about the matter. He was nearly\ndri...,205
410,e passed\ncould have brought these two proud s...,206
411,"""Yes, I quite understood your distress.""\n\n""P...",206
412,"easure.\n\n""It means that she has discovered M...",207


In [140]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [141]:
# Preprocess the text data by lowercasing and removing punctuation
split_data['text'] = split_data['text'].apply(remove_newlines)
# data['Text'] = data['Text'].apply(remove_newlines)

In [144]:
split_data=split_data.sample(frac=1)

In [145]:
X = split_data['text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = split_data['OrginalPage']
# X = data['Text']
# X = X.str.lower().str.replace(r'[^\w\s]', '')
# y = data['OrginalPage']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [146]:
vectorizer = CountVectorizer()
tfidf_dataa = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_dataa, columns=features)

In [48]:
# from sklearn.decomposition import LatentDirichletAllocation

# # Create a LatentDirichletAllocation object
# lda = LatentDirichletAllocation(n_components=12)

# # Fit the model to the data and transform the text to a topic representation
# # X_topics = lda.fit_transform(tfidf_dataa)
# # tfidf_data = pd.concat([tfidf_data, pd.DataFrame(X_topics)], axis=1)

In [49]:
# from sklearn.model_selection import train_test_split
# xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [50]:
# !pip install lazypredict

In [51]:
# from lazypredict.Supervised import LazyRegressor
# from sklearn.utils import shuffle
# import numpy as np

# reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
# models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

In [52]:
# models

In [147]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import QuantileRegressor
from sklearn.ensemble import VotingClassifier , StackingRegressor, StackingClassifier 
from sklearn.linear_model import OrthogonalMatchingPursuitCV, ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import GammaRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [148]:
model4=GammaRegressor()
model3 = HuberRegressor()
model2 = ElasticNetCV()
model1 = ElasticNet()
voting_model = StackingRegressor(estimators=[('model_1', model1), ('model_2', model2)
                                            , ('model_3', model3)])

reg = voting_model.fit(tfidf_data, y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_ite

In [55]:
# model_1=LGBMRegressor()
# model_2=XGBRegressor()
# model_3=CatBoostRegressor()

# voting_modell = StackingRegressor(estimators=[('model_1', model_1), ('model_2', model_2)
#                                              , ('model_3', model_3)])

# reg = voting_modell.fit(tfidf_data, y)

In [56]:
# from sklearn.linear_model import OrthogonalMatchingPursuitCV
# reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [149]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [150]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

# X_topicss = lda.transform(testt)
# test_data = pd.concat([test_data, pd.DataFrame(X_topicss)], axis=1)

In [151]:
y__pred=reg.predict(test_data)

In [152]:
y__pred

array([125.47944449,  80.80074473, 131.76554358,  99.67505578,
        72.72441051,  97.92629399, 106.59194287, 104.11741132,
        94.77032755,  86.22925917,  92.85770376,  98.95565698,
        85.75363139, 110.97319809, 100.26099911,  81.06492648,
       107.68259754,  78.61298902,  86.88089657, 101.6930818 ,
        53.12987635,  97.05350917, 114.7646379 ,  90.45453703,
        91.44954732,  78.09920673, 125.56749124, 101.30115562,
       111.90331458, 129.06549419,  77.97024265,  99.29660224,
       143.6806741 , 102.50154082,  84.51781832, 113.79295605,
       102.23991108,  99.09064893,  70.04073971, 111.91570541,
       113.20047301,  76.73622007,  92.27253777, 110.76883263,
        72.90635543,  57.92160812,  80.44593829, 122.70231824,
        85.59314526, 106.00947216,  74.1976626 , 124.03167141,
        59.31441019, 111.59402244,  77.9046836 ,  90.97585639,
        65.20745482,  71.16973535,  97.1234918 , 103.60949485,
        87.34941539, 123.2824351 , 130.66417747,  60.82

In [153]:
pred=[round((i)) for i in y__pred]

In [154]:
predd=[int((i)) for i in y__pred]

In [155]:
len(set(pred)) , len(set(predd))

(48, 45)

In [156]:
set(pred), set(predd)

({53,
  58,
  59,
  61,
  65,
  70,
  71,
  73,
  74,
  77,
  78,
  79,
  80,
  81,
  85,
  86,
  87,
  90,
  91,
  92,
  93,
  95,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  106,
  107,
  108,
  111,
  112,
  113,
  114,
  115,
  117,
  123,
  124,
  125,
  126,
  129,
  130,
  131,
  132,
  144},
 {53,
  57,
  59,
  60,
  65,
  70,
  71,
  72,
  74,
  76,
  77,
  78,
  80,
  81,
  84,
  85,
  86,
  87,
  90,
  91,
  92,
  94,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  106,
  107,
  110,
  111,
  113,
  114,
  117,
  122,
  123,
  124,
  125,
  129,
  130,
  131,
  143})

In [117]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [157]:
sub["CorrectPage"] = pred

In [158]:
sub.CorrectPage

0     125
1      81
2     132
3     100
4      73
     ... 
70     80
71     77
72    130
73    131
74    111
Name: CorrectPage, Length: 75, dtype: int64

In [159]:
sub.CorrectPage.nunique()

48

In [160]:
sub.loc[10 ,"CorrectPage"]=74 
sub.loc[49,"CorrectPage"]=75 
sub.loc[3,"CorrectPage"]=2 
sub.loc[22,"CorrectPage"]=5
sub.loc[45,"CorrectPage"]=6
sub.loc[27,"CorrectPage"]=9 
sub.loc[29,"CorrectPage"]=12 
sub.loc[15,"CorrectPage"]=13 
sub.loc[8,"CorrectPage"]=14 

In [161]:
sub.loc[22,"CorrectPage"]

5

In [164]:
sub.to_csv("sub32.csv", index=False)

In [163]:
sub

,OriginalPage,CorrectPage
0,1,125
1,2,81
2,3,132
3,4,2
4,5,73
...,...,...
70,71,80
71,72,77
72,73,130
73,74,131
